In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import warnings
warnings.filterwarnings('ignore')
import json 
with open('Intent.json') as json_file:
    data = json.load(json_file)
data = data['intents']

In [3]:
dataset = pd.DataFrame(columns=['intent', 'text', 'response'])
for i in data:
    intent = i['intent']
    for t, r in zip(i['text'], i['responses']):
        row = {'intent': intent, 'text': t, 'response':r}
        dataset = dataset.append(row, ignore_index=True)
dataset

,intent,text,response
0,Greeting,Hi,"Hi human, please tell me your GeniSys user"
1,Greeting,Hi there,"Hello human, please tell me your GeniSys user"
2,Greeting,Hola,"Hola human, please tell me your GeniSys user"
3,GreetingResponse,My user is Adam,Great! Hi <HUMAN>! How can I help?
4,GreetingResponse,This is Adam,"Good! Hi <HUMAN>, how can I help you?"
...,...,...,...
76,PodBayDoorResponse,Why can you not open the pod bay door,It's life Jim but not as we know it!
77,PodBayDoorResponse,Why will you not open the pod bay door,System says no!
78,SelfAware,Can you prove you are self-aware,"That is an interesting question, can you prove..."
79,SelfAware,Can you prove you are self aware,"That is an difficult question, can you prove t..."


In [4]:
def cosine_distance_countvectorizer_method(s1, s2):
    
    # sentences to list
    allsentences = [s1 , s2]
    
    # packages
    from sklearn.feature_extraction.text import CountVectorizer
    from scipy.spatial import distance
    
    # text to vector
    vectorizer = CountVectorizer()
    all_sentences_to_vector = vectorizer.fit_transform(allsentences)
    text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()
    
    # distance of similarity
    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    return round((1-cosine),2)

In [5]:
def respond(text):
    maximum = float('-inf')
    response = ""
    closest = ""
    for i in dataset.iterrows():
        sim = cosine_distance_countvectorizer_method(text, i[1]['text'])
        if sim > maximum:
            maximum = sim
            response = i[1]['response']
            closest = i[1]['text']
    return response

In [6]:
while True:
    text = str(input("Input: "))
    if text.lower() == "exit":
        print("Response: Exiting.....")
        break
    print("Response:",respond(text))

Input: hi
Response: Hi human, please tell me your GeniSys user
Input: are
Response: Hello, I am great, how are you? Please tell me your GeniSys user
Input: male
Response: Hi human, please tell me your GeniSys user
Input: user
Response: Great! Hi <HUMAN>! How can I help?
Input: home
Response: Hi human, please tell me your GeniSys user
Input: exit
Response: Exiting.....
